In [8]:
# install surprise: conda install -c conda-forge scikit-surprise
import os
import numpy as np
import surprise
import pandas as pd
import surprise
from surprise import Reader, Dataset

# numpy로 데이터 읽어오기

In [9]:
data = np.genfromtxt("./ml-25m/ratings.csv", delimiter=",", names=True)

len(data), data, data.shape

(25000095,
 array([(1.00000e+00,   296., 5. , 1.14788004e+09),
        (1.00000e+00,   306., 3.5, 1.14786882e+09),
        (1.00000e+00,   307., 5. , 1.14786883e+09), ...,
        (1.62541e+05, 56176., 2. , 1.24095070e+09),
        (1.62541e+05, 58559., 4. , 1.24095343e+09),
        (1.62541e+05, 63876., 5. , 1.24095252e+09)],
       dtype=[('userId', '<f8'), ('movieId', '<f8'), ('rating', '<f8'), ('timestamp', '<f8')]),
 (25000095,))

In [10]:
# timestamp 기준으로 나누기
train_data = np.array([list(i) for i in data if 1104505203 <= i[3] <= 1230735592])
test_data = np.array([list(i) for i in data if i[3] >= 1230735600])
print(train_data)

[[1.00000000e+00 2.96000000e+02 5.00000000e+00 1.14788004e+09]
 [1.00000000e+00 3.06000000e+02 3.50000000e+00 1.14786882e+09]
 [1.00000000e+00 3.07000000e+02 5.00000000e+00 1.14786883e+09]
 ...
 [1.62521000e+05 5.96150000e+04 3.50000000e+00 1.21933944e+09]
 [1.62521000e+05 5.97250000e+04 4.00000000e+00 1.21933939e+09]
 [1.62521000e+05 6.00740000e+04 4.00000000e+00 1.21933941e+09]]


# 이후로 LightFM or Surprise package 사용해서 구현해보기

In [ ]:
user_id = np.unique(np.array([i[0] for i in train_data]))  # (27915,))
movie_id = np.unique(np.array([i[1] for i in train_data]))  # (10533,))
rating = np.array([i[2] for i in train_data])

In [ ]:
from collections import defaultdict

rating_dict = defaultdict(dict)
for i in train_data:
    rating_dict[i[0]][i[1]] = i[2]

In [ ]:
# movie_id의 id가 각 키의 values에 없다면, movie-id를 추가하고 값을 0으로 넣는다. 이 과정을 거치면 fillna(0)과 같은 효과
# 그리고 추가된 rating_dict에서 key를 row, values flatten하게 쌓으면된다.
# user_id (27915,))
# movie_id (10533,))
for i in range(len(user_id)):
    for j in range(len(movie_id)):
        if movie_id[j] not in rating_dict[user_id[i]].keys():
            rating_dict[user_id[i]][movie_id[j]] = 0.0
        else:
            pass

In [ ]:
# # rating_dict의 키(user_id)마다 movie_id를 기준으로 내림차순 정렬, matrix로 재구성
# rating_matrix = []
# for i in range(len(rating_dict)):
#     sorted(rating_dict[user_id[i]])

# rating_dict에서 movie의 개수가 같은 지 확인

In [ ]:
len(rating_dict[1])  == len(rating_dict[2]) # 10533

In [ ]:
rating_dic = {
    'userId': [],
    'movieId': [],
    'rating': []
}

id_list = []
mov_set = set()

In [ ]:
for user_key in rating_dict.keys():
    for mov_key in rating_dict[user_key]:
        rating_dic["userId"].append(user_key)
        rating_dic["movieId"].append(mov_key)
        rating_dic["rating"].append(rating_dict[user_key][mov_key])

print(len(rating_dic["userId"]))
print(len(rating_dic["movieId"]))
print(len(rating_dic["rating"]))

In [ ]:
train_df = pd.DataFrame(rating_dic)
train_df

In [ ]:
reader = surprise.Reader(rating_scale=(1,5))
reader

In [ ]:
col_list = ["userId", "movieId", "rating"]

train_data = surprise.Dataset.load_from_df(train_df[col_list], reader)

trainset = train_data.build_full_trainset()

algo = surprise.SVD()
algo.fit(trainset)

# Testset

In [ ]:
from collections import defaultdict

In [ ]:
test_rating_dict = defaultdict(dict)
for i in test_data:
    test_rating_dict[i[0]][i[1]] = i[2]

In [ ]:
for i in range(len(user_id)):
    for j in range(len(movie_id)):
        if movie_id[j] not in test_rating_dict[user_id[i]].keys():
            test_rating_dict[user_id[i]][movie_id[j]] = 0.0
        else:
            pass

In [ ]:
test_rating_dic = {
    'userId': [],
    'movieId': [],
    'rating': []
}

id_list = []
mov_set = set()

In [ ]:
for user_key in test_rating_dict.keys():
    for mov_key in test_rating_dict[user_key]:
        test_rating_dic["userId"].append(user_key)
        test_rating_dic["movieId"].append(mov_key)
        test_rating_dic["rating"].append(test_rating_dict[user_key][mov_key])

In [ ]:
test_df = pd.DataFrame(test_rating_dic)
test_df

In [ ]:
test_data = surprise.Dataset.load_from_df(test_df[col_list], reader)
testset = test_data.build_full_trainset()

testset = testset.build_testset()
predictions = algo.test(testset)

from surprise import accuracy
accuracy.rmse(predictions)

test_df

# 너무 커서 돌아가지 않음